# Data Clean

In [1]:
### Import packages
import pandas as pd
from geopy.geocoders import Nominatim
import geojson
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import seaborn as sns
import plotnine
from plotnine import *

In [10]:
## Introduce the final dataset to enact some calculations
dfb = pd.read_csv('/Users/leowu/Desktop/Intern--IPS/Boston Housing/Dataset/Final_df_Boston.csv')
dfb.head()

,Unnamed: 0,Date,Type,Town,Doc Number,Price,Street,Address Description,Grantee Name(s),is_corp,latitude,longitude,neighborhood,Month
0,0,2022-01-03,DEED,BOSTON,116,462500.0,KEITH ST 14,,R BROOKS INC,True,42.288128,-71.166939,West Roxbury,1
1,1,2022-01-03,DEED,BOSTON,146,800000.0,DEER ST 12-14,DORCHESTER,NaN,True,42.312397,-71.058040,Dorchester,1
2,2,2022-01-03,DEED,BOSTON,153,1155000.0,ST ANDREW RD 92-94,EAST BOSTON,"CHEN TONY,THEA SOK CHENG",False,42.384151,-71.001217,East Boston,1
3,3,2022-01-03,DEED,BOSTON,198,1115000.0,HARLEM ST 20,DORCHESTER,"CORDERO SANZ HENRY M,CORDERO HENRY G",False,42.300333,-71.081231,Mattapan,1
4,4,2022-01-03,DEED,BOSTON,210,810000.0,LONDON ST 101,EAST BOSTON,"LI JINLIN,LI IVY",False,42.372613,-71.039454,East Boston,1


In [3]:
# Check the NA value
dfb.isna().sum()

Unnamed: 0              0
Date                    0
Type                    0
Town                    0
Doc Number              0
Price                   0
Street                  0
Address Description     0
Grantee Name(s)        29
is_corp                 0
latitude                0
longitude               0
neighborhood            0
Month                   0
dtype: int64

In [4]:
dfb.Price.describe()

count    7.711000e+03
mean     1.297359e+06
std      4.831514e+06
min      1.000000e+05
25%      5.800000e+05
50%      7.900000e+05
75%      1.180500e+06
max      1.878550e+08
Name: Price, dtype: float64

In [5]:
# Check the proportion of investors among DEED housings
dfb[(dfb.Price >= 2000000) & (dfb.is_corp == True) & (dfb.Type == 'DEED')].shape[0]/\
dfb[(dfb.Price >= 2000000) & (dfb.Type == 'DEED')].shape[0]

0.6802507836990596

In [6]:
# Check the proportion of investors among UNIT DEED housings
dfb[(dfb.Price >= 2000000) & (dfb.is_corp == True) & (dfb.Type == 'UNIT DEED')].shape[0]/\
dfb[(dfb.Price >= 2000000) & (dfb.Type == 'UNIT DEED')].shape[0]

0.37731958762886597

In [7]:
dfb.shape

(7711, 14)

In [9]:
# Exclude those neighborhood is unknown
dfb_new= dfb[dfb.neighborhood!='Unknown']
dfb_new.shape

(7706, 14)

### Do some calculations on a neighborhood-level

In [11]:
## Generate a list containing all neighborhood names
lis = dfb_new.neighborhood.unique().tolist()
lis

['West Roxbury',
 'Dorchester',
 'East Boston',
 'Mattapan',
 'Allston',
 'Roslindale',
 'Roxbury',
 'Bay Village',
 'Jamaica Plain',
 'South Boston',
 'Hyde Park',
 'Brighton',
 'Beacon Hill',
 'North End',
 'Back Bay',
 'Mission Hill',
 'Charlestown',
 'Downtown',
 'Chinatown',
 'Fenway',
 'South End',
 'Leather District',
 'South Boston Waterfront',
 'West End',
 'Longwood Medical Area']

In [12]:
for i in lis:
    total = dfb_new[dfb_new.neighborhood == i].Price.sum()
    print(i,':',total)

West Roxbury : 302433218.0
Dorchester : 696606882.55
East Boston : 519022430.03999996
Mattapan : 138829539.78
Allston : 288923812.0
Roslindale : 262314464.0
Roxbury : 294647805.55
Bay Village : 62572500.0
Jamaica Plain : 465020317.01
South Boston : 1008410576.93
Hyde Park : 412316944.20000005
Brighton : 455213570.5
Beacon Hill : 339683786.5
North End : 176658250.0
Back Bay : 953853048.49
Mission Hill : 133169200.0
Charlestown : 507289139.05
Downtown : 1391580610.1
Chinatown : 216707200.0
Fenway : 185010737.3
South End : 742359535.0
Leather District : 1341000.0
South Boston Waterfront : 407474054.33
West End : 25908250.0
Longwood Medical Area : 12961767.0


In [13]:
dfb_new[dfb_new.is_corp==True].Price.sum()*0.05

208723612.364

In [14]:
# Introduce a new df containing grantors' names
out = pd.read_excel('/Users/leowu/Desktop/Intern--IPS/Boston Housing/Dataset/Grantors above $2M.xlsx')
out.head()

,Date,Type,Town,Doc Number,Price,Street,Address Description,Grantor,Grantee Name(s),is_corp,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,2022-12-15,UNIT DEED,BOSTON,100076,4250000.0,SEAPORT BLVD UNIT 9D 150,,150 SEAPORT LLC,HARBOR 10A LLC,True,NaN,NaN,NaN
1,2022-03-31,DEED,BOSTON,27013,3550000.0,HIGH ST 29,CHARLESTOWN,A KIM SAAL REVOCABLE TRUST,"MURPHY JOSEPH G,MURPHY REBECCA",False,NaN,NaN,0.606211
2,2022-03-15,DEED,BOSTON,21931,2320000.0,STRATHMORE RD 59A,BRIGHTON,A LIMITED LIABILITY COMPANY LLC,"WU HANWEN,ZHOU YANG",False,NaN,NaN,NaN
3,2022-04-01,UNIT DEED,BOSTON,27794,2500000.0,BEACON ST UNIT 1 94,,A SMOKI BACON 1994 REVOCABLE TRUST,SOMERSET VENTURES LLC,True,NaN,NaN,NaN
4,2022-04-22,UNIT DEED,BOSTON,34260,3650000.0,HANSON ST UNIT 2 1,,A&T STAPLETON REALTY TRUST,"COX GERARD L,COX NICOLA S",False,NaN,NaN,NaN


In [15]:
df1 = dfb_new[dfb_new.Price>=2000000]
dff = pd.merge(df1,out[['Type','Doc Number','Grantor']],how='left',on=['Type','Doc Number'])
dff.head()

,Unnamed: 0,Date,Type,Town,Doc Number,Price,Street,Address Description,Grantee Name(s),is_corp,latitude,longitude,neighborhood,Month,Grantor
0,10,2022-01-04,DEED,BOSTON,778,2050000.0,ADDISON ST 95,EAST BOSTON,ADDISON LLC 95-97,True,42.385574,-71.017815,East Boston,1,GADGETZAN LLC\t
1,20,2022-01-06,DEED,BOSTON,1391,2475000.0,WINFIELD ST 7,SOUTH BOSTON,WINFIELD LLC 7,True,42.331879,-71.042732,South Boston,1,QB WINFIELD LLC
2,37,2022-01-07,DEED,BOSTON,2066,5550000.0,MT VERNON ST 59,,TVM CAPITAL HEALTHCARE PARTNERS INC,True,42.358553,-71.066143,Beacon Hill,1,NaN
3,40,2022-01-07,DEED,BOSTON,2138,2150000.0,CLARK ST 9-11,,CLARK STREET LLC 9-11,True,42.365022,-71.052070,North End,1,NaN
4,49,2022-01-12,DEED,BOSTON,3141,6000000.0,MARLBOROUGH ST 122,,"BRYN MAWR TRUST COMPANY OF DELAWARE,EADDY ROBE...",True,42.353017,-71.077250,Back Bay,1,"BRYN MAWR TRUST COMPANY OF DELAWARE, MMK 2007 ..."


In [17]:
# Judge whether the grantors are investor
dff['is_invest'] = ~dff.Grantor.isna()

In [18]:
dff.is_invest.sum()

490

In [19]:
for i in lis:
    total = dff[dff.neighborhood == i].is_invest.sum()
    print(i,':',total)

West Roxbury : 9
Dorchester : 15
East Boston : 13
Mattapan : 2
Allston : 5
Roslindale : 3
Roxbury : 12
Bay Village : 10
Jamaica Plain : 10
South Boston : 45
Hyde Park : 5
Brighton : 14
Beacon Hill : 21
North End : 20
Back Bay : 57
Mission Hill : 16
Charlestown : 23
Downtown : 85
Chinatown : 6
Fenway : 13
South End : 57
Leather District : 0
South Boston Waterfront : 46
West End : 1
Longwood Medical Area : 2


In [20]:
df2 = dfb_new[dfb_new.is_corp==True]
df2.groupby('neighborhood').Type.count()

neighborhood
Allston                     32
Back Bay                    96
Bay Village                 11
Beacon Hill                 43
Brighton                    61
Charlestown                 67
Chinatown                   10
Dorchester                 183
Downtown                   197
East Boston                145
Fenway                      38
Hyde Park                   46
Jamaica Plain               76
Longwood Medical Area        1
Mattapan                    42
Mission Hill                24
North End                   47
Roslindale                  38
Roxbury                     77
South Boston               155
South Boston Waterfront     44
South End                   74
West End                     4
West Roxbury                43
Name: Type, dtype: int64

In [21]:
for i in lis:
    df2 = dfb_new[dfb_new.is_corp==True]
    total = df2[df2.neighborhood == i].Price.sum() *0.05
    print(i,':',total)

West Roxbury : 3410513.85
Dorchester : 10387080.15
East Boston : 8772090.5
Mattapan : 1960148.35
Allston : 11370464.4
Roslindale : 2262418.0500000003
Roxbury : 5386540.15
Bay Village : 1240025.0
Jamaica Plain : 4738190.14
South Boston : 22105534.9325
Hyde Park : 14147278.75
Brighton : 12869025.0
Beacon Hill : 6803475.0
North End : 5051212.5
Back Bay : 30743925.0
Mission Hill : 3953300.0
Charlestown : 6538961.625
Downtown : 24229628.900000002
Chinatown : 9094750.0
Fenway : 4423373.75
South End : 8902428.1
Leather District : 0.0
South Boston Waterfront : 9728248.2165
West End : 574000.0
Longwood Medical Area : 31000.0


##### 2% Tax

In [25]:
df1 = dfb_new[dfb_new.Price>=2000000]
df1['Over_2M'] = df1.Price - 2000000
df1['Tax'] = df1.Over_2M * 0.02
df1.head()

/var/folders/39/p0rpjrxj68l4cbrkhyvw78fm0000gn/T/ipykernel_92216/3701719970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/39/p0rpjrxj68l4cbrkhyvw78fm0000gn/T/ipykernel_92216/3701719970.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,Date,Type,Town,Doc Number,Price,Street,Address Description,Grantee Name(s),is_corp,latitude,longitude,neighborhood,Month,Over_2M,Tax
10,10,2022-01-04,DEED,BOSTON,778,2050000.0,ADDISON ST 95,EAST BOSTON,ADDISON LLC 95-97,True,42.385574,-71.017815,East Boston,1,50000.0,1000.0
20,20,2022-01-06,DEED,BOSTON,1391,2475000.0,WINFIELD ST 7,SOUTH BOSTON,WINFIELD LLC 7,True,42.331879,-71.042732,South Boston,1,475000.0,9500.0
37,37,2022-01-07,DEED,BOSTON,2066,5550000.0,MT VERNON ST 59,,TVM CAPITAL HEALTHCARE PARTNERS INC,True,42.358553,-71.066143,Beacon Hill,1,3550000.0,71000.0
40,40,2022-01-07,DEED,BOSTON,2138,2150000.0,CLARK ST 9-11,,CLARK STREET LLC 9-11,True,42.365022,-71.052070,North End,1,150000.0,3000.0
49,49,2022-01-12,DEED,BOSTON,3141,6000000.0,MARLBOROUGH ST 122,,"BRYN MAWR TRUST COMPANY OF DELAWARE,EADDY ROBE...",True,42.353017,-71.077250,Back Bay,1,4000000.0,80000.0


In [26]:
for i in lis:
    total = df1[df1.neighborhood == i].Tax.sum()
    print(i,':',total)

West Roxbury : 676950.0
Dorchester : 582558.4
East Boston : 446300.0
Mattapan : 154600.0
Allston : 3884325.7600000002
Roslindale : 321625.0
Roxbury : 607828.0054
Bay Village : 267640.0
Jamaica Plain : 157287.0
South Boston : 5165111.973
Hyde Park : 4807327.0
Brighton : 3856300.0
Beacon Hill : 2781050.0
North End : 814615.0
Back Bay : 11056152.0
Mission Hill : 1025400.0
Charlestown : 1272556.0
Downtown : 6645489.06
Chinatown : 3287100.0
Fenway : 1060380.0
South End : 3046162.1
Leather District : 0.0
South Boston Waterfront : 3038639.6
West End : 140000.0
Longwood Medical Area : 89735.34


##### 2/4% Mixture Tax

In [23]:
df1 = dfb_new[dfb_new.Price >= 2000000].reset_index(drop=True)

df1['Over_2M'] = 0
df1['Over_4M'] = 0

for i in range(df1.shape[0]):
    if df1.Price[i] >= 4000000:
        df1['Over_4M'][i] = df1.Price[i] - 4000000
        df1['Over_2M'][i] = 2000000
    else:
        df1['Over_2M'][i] = df1.Price[i] - 2000000

df1['Tax1'] = df1.Over_2M * 0.02
df1['Tax2'] = df1.Over_4M * 0.04

df1.head()

/var/folders/39/p0rpjrxj68l4cbrkhyvw78fm0000gn/T/ipykernel_92216/820485914.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/39/p0rpjrxj68l4cbrkhyvw78fm0000gn/T/ipykernel_92216/820485914.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/39/p0rpjrxj68l4cbrkhyvw78fm0000gn/T/ipykernel_92216/820485914.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,Date,Type,Town,Doc Number,Price,Street,Address Description,Grantee Name(s),is_corp,latitude,longitude,neighborhood,Month,Over_2M,Over_4M,Tax1,Tax2
0,10,2022-01-04,DEED,BOSTON,778,2050000.0,ADDISON ST 95,EAST BOSTON,ADDISON LLC 95-97,True,42.385574,-71.017815,East Boston,1,50000.0,0.0,1000.0,0.0
1,20,2022-01-06,DEED,BOSTON,1391,2475000.0,WINFIELD ST 7,SOUTH BOSTON,WINFIELD LLC 7,True,42.331879,-71.042732,South Boston,1,475000.0,0.0,9500.0,0.0
2,37,2022-01-07,DEED,BOSTON,2066,5550000.0,MT VERNON ST 59,,TVM CAPITAL HEALTHCARE PARTNERS INC,True,42.358553,-71.066143,Beacon Hill,1,2000000.0,1550000.0,40000.0,62000.0
3,40,2022-01-07,DEED,BOSTON,2138,2150000.0,CLARK ST 9-11,,CLARK STREET LLC 9-11,True,42.365022,-71.052070,North End,1,150000.0,0.0,3000.0,0.0
4,49,2022-01-12,DEED,BOSTON,3141,6000000.0,MARLBOROUGH ST 122,,"BRYN MAWR TRUST COMPANY OF DELAWARE,EADDY ROBE...",True,42.353017,-71.077250,Back Bay,1,2000000.0,2000000.0,40000.0,80000.0


In [24]:
for i in lis:
    df2 = df1[df1.neighborhood == i]
    total = df2.Tax1.sum()+df2.Tax2.sum()
    print(i,':',total)

West Roxbury : 1120950.0
Dorchester : 770058.4
East Boston : 706800.0
Mattapan : 208600.0
Allston : 7602851.5200000005
Roslindale : 519250.0
Roxbury : 888208.0108
Bay Village : 285640.0
Jamaica Plain : 157287.0
South Boston : 9329725.946
Hyde Park : 9453354.0
Brighton : 7353800.0
Beacon Hill : 4176350.0
North End : 1092730.0
Back Bay : 19564658.6666
Mission Hill : 1699400.0
Charlestown : 1821596.0
Downtown : 10084058.760000002
Chinatown : 6342150.0
Fenway : 1701280.0
South End : 3560442.1
Leather District : 0.0
South Boston Waterfront : 4600074.6
West End : 240000.0
Longwood Medical Area : 98970.68


In [27]:
dfb_new[dfb_new.Price>=4000000].groupby('neighborhood').Type.count()

neighborhood
Allston                     3
Back Bay                   48
Bay Village                 1
Beacon Hill                23
Brighton                    8
Charlestown                 8
Chinatown                   5
Dorchester                  5
Downtown                   31
East Boston                 3
Fenway                      5
Hyde Park                   3
Jamaica Plain               1
Longwood Medical Area       1
Mattapan                    2
Mission Hill                6
North End                   6
Roslindale                  3
Roxbury                     5
South Boston                8
South Boston Waterfront    25
South End                  22
West End                    1
West Roxbury                4
Name: Type, dtype: int64